# Data Collection (Guns)

---

In [ ]:
# Module check/download

try:
    from pip import main as pip_main
except:
    from pip._internal import main as pip_main

def import_or_install(package):
    
    i = package[0]
    
    try:
        __import__(package[0])
        print(i+": "+"ok")
        
    except ImportError:
        print("\n"+i+": "+"installing...")
        pip_main(['install', package[1]])
        print(i+": "+"ok")

modules = [('glob','glob'), ('matplotlib','matplotlib'), 
           ('natsort','natsort'), ('cv2','opencv-python')]

for i in modules:
    import_or_install(i)

In [ ]:
# Relevant libraries

import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import zipfile
import natsort
import urllib
import glob
import cv2
import os

In [ ]:
# Data retrieval (Desktop will be used as workspace)

def get_data():
    
    path = "https://sci2s.ugr.es/sites/default/files/files/TematicWebSites/WeaponsDetection/BasesDeDatos/"
    files = ["WeaponS.zip", "WeaponS_bbox.zip"]
    labels = ['images', 'bounding boxes']
    
    for i in range(len(files)):
        
        url = path+files[i]
        
        print ("\ndownloading {}...".format(labels[i]))    
        filename, headers = urllib.request.urlretrieve(
            url,filename=os.path.expanduser(str("~/Desktop/{}").format(files[i])))    
        print ("download complete!")
        
        os.chdir(os.path.expanduser("~/Desktop/"))
        print ("\nextracting {}...".format(labels[i]))
        zipf = zipfile.ZipFile(os.path.expanduser(str("~/Desktop/{}").format(files[i])), "r")
        zipf.extractall()
        zipf.close()
        print ("extract complete!")
    
get_data()

In [ ]:
# Data prep for RetinaNet

files = glob.glob(os.path.expanduser("~/Desktop/WeaponS/*.jpg"))
files = natsort.natsorted(files)

annotations = glob.glob(os.path.expanduser("~/Desktop/WeaponS_bbox/*.xml"))
annotations = natsort.natsorted(annotations)

###
#https://stackoverflow.com/questions/53317592/reading-pascal-voc-annotations-in-python
def read_content(xml_file: str):

    tree = ET.parse(xml_file)
    root = tree.getroot()
    list_with_all_boxes = []

    for boxes in root.iter('object'):

        filename = root.find('filename').text
        ymin, xmin, ymax, xmax = None, None, None, None

        for box in boxes.findall("bndbox"):
            ymin = int(box.find("ymin").text)
            xmin = int(box.find("xmin").text)
            ymax = int(box.find("ymax").text)
            xmax = int(box.find("xmax").text)

        list_with_single_boxes = [xmin, ymin, xmax, ymax]
        list_with_all_boxes.append(list_with_single_boxes)

    return filename, list_with_all_boxes
###

store = []

for k in range(len(files)):
    
    name, boxes = read_content(annotations[k])
        
    if len(boxes) == 1:
        
        rec = [files[k]]
        rec.extend(boxes[0])
        rec.append('gun')
        store.append(rec)
            
    else:
        for c in range(len(boxes)):
            
            rec = [files[k]]
            rec.extend(boxes[c])
            rec.append('gun')
            store.append(rec)

#for rec in store:
#    print(", ".join(str(x) for x in rec))

In [ ]:
# Sanity check

img = cv2.imread(store[0][0],1)
cv2.rectangle(img,(store[0][1], store[0][2]), (store[0][3], store[0][4]),(0,255,0),3)
cv2.imshow('image',img)
cv2.waitKey(0)

In [ ]:
# Save in RetinaNet format

file = open(os.path.expanduser("~/Desktop/pistolData.txt"),'w')

for rec in store:
    file.write(", ".join(str(x) for x in rec)+'\n')
    
file.close()